# Validation

In [ ]:
using FindSteadyStates
using CairoMakie
import RetroSignalModel as rs

In [ ]:
rtgM4 = rs.rtgM4(1);
pr_table = rs.get_protein_lookup(rtgM4);

## Initiate protein concentrations

- Get expression levels: `getExpLevels` uses data from [GSE102475](https://github.com/NTUMitoLab/RetroSignalModel.jl?organization=NTUMitoLab&organization=NTUMitoLab#differential-gene-expression)
- Sampling agent concentrations: The [`init_u`](https://github.com/stevengogogo/RetroSignalModel.jl/blob/73ffb3713b8e5481a85c3ce9590c9c53e8440faf/src/models/utils.jl#L63-L85) can setup protein concentrations with given total expression levels.

In [ ]:
# Set expression level
exp = rs.getExpLevels()
# Knock out Bmh
exp_dBmh = (exp..., Bmh=1e-4)

In [ ]:
# Get agent concentrations
u_low = rs.init_u(rtgM4.model, pr_table; expLevels=exp_dBmh, init_s=0.2);

In [ ]:
# Get solution
sol = rs.getSteadySol(rtgM4, u_low)

In [ ]:
r1 = rs.getOutput(sol, "Rtg1", pr_table)
@show r1.Conc_Cyt

In [ ]:
function measure_trans(m, p, dmg_sig, explvl, gfp)
    pr_table = rs.get_protein_lookup(m)
    u = rs.init_u(m.model, pr_table; expLevels= explvl, init_s=dmg_sig);
    prob = DEsteady(func=m.model, u0=u, p=p, method=rs.SSMETHOD)
    sol = solve(prob)
    return rs.getOutput(sol, gfp, pr_table)
end

In [ ]:
measure_trans(rtgM4, rtgM4.p, 0.4, exp, "Rtg3")

In [ ]:
measure_trans(rtgM4, rtgM4.p, 0.1, exp, "Rtg3")
measure_trans(rtgM4, rtgM4.p, 0.9, (exp..., Bmh=1e-4), "Rtg3")

In [ ]:
tbl = (x = [1, 1, 1, 1,2,2,2,2],
       height = [
        measure_trans(rtgM4, rtgM4.p, 0.1, exp, "Rtg3").Conc_Cyt,
        measure_trans(rtgM4, rtgM4.p, 0.1, exp, "Rtg3").Conc_Nuc,
        measure_trans(rtgM4, rtgM4.p, 0.9, exp, "Rtg3").Conc_Cyt,
        measure_trans(rtgM4, rtgM4.p, 0.9, exp, "Rtg3").Conc_Nuc,
        measure_trans(rtgM4, rtgM4.p, 0.1, (exp..., Bmh=1), "Rtg3").Conc_Cyt,
        measure_trans(rtgM4, rtgM4.p, 0.1, (exp..., Bmh=1), "Rtg3").Conc_Nuc,
        measure_trans(rtgM4, rtgM4.p, 0.9, (exp..., Bmh=1), "Rtg3").Conc_Cyt,
        measure_trans(rtgM4, rtgM4.p, 0.9, (exp..., Bmh=1), "Rtg3").Conc_Nuc
       ],
       grp = [1, 1, 2, 2, 3, 3, 4, 4], # Genotype
       grp1 = [1, 2, 1, 2, 1, 2, 1, 2] # Mitochondrial condition
)

In [ ]:
fig = Figure()
colors = Makie.wong_colors()
ax = Axis(fig[1,1], xticks = (1:2, ["Control (WT/mtDamage)", "Bmh-del (WT/mtDamage)"]),
title = "Rtg3 translocation")

barplot!(ax, tbl.x, tbl.height,
        dodge = tbl.grp,
        stack = tbl.grp1,
        color = colors[tbl.grp1]
)

# Legend
labels = ["Rtg3-GFP (Cytosol)", "Rtg3-GFP (Nucleus)"]
elements = [PolyElement(polycolor = colors[i]) for i in 1:length(labels)]
Legend(fig[1,2], elements, labels, "Rtg3 Concentration")
# save("validation.pdf", fig)
fig